<a href="https://colab.research.google.com/github/vicmcl/e-commerce/blob/simulation/notebook_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from google.colab import drive
import sys

content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.flush_and_unmount()
drive.mount(str(drive_path))

sys.path.append(str(drive_path / "My Drive" / "OCR" / "utils"))

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [37]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

%matplotlib inline
sns.set_style('white')

In [109]:
df = pd.read_json(drive_path / "My Drive" / "OCR" / "4_Olist" / "orders.json").sort_values("time_since_order")

In [110]:
df[["time_since_order", "delay"]] = df[["time_since_order", "delay"]].applymap(lambda x: pd.Timedelta(x, unit="ms"))

In [122]:
train_set_age = range(2, 7)
train_set_periods = [2, 4, 6]

In [115]:
six_months = pd.Timedelta(180, unit='D')
train_sets = dict()

for i in train_set_periods:
    train_period = pd.Timedelta(i * 30, unit='D')
    df_train = df[df["time_since_order"] > six_months]
    df_train = df_train[df_train["time_since_order"] < train_period + six_months]
    train_sets[f"{i} months"] = df_train

In [123]:
test_sets = dict()
test_set_length = 2 # months

for j in train_set_age:
    start_time = pd.Timedelta(180, unit='D') - pd.Timedelta((j - 2) * 30, unit='D')
    end_time = start_time - pd.Timedelta(test_set_length * 30, unit='D')
    df_test = df[df["time_since_order"] < start_time]
    df_test = df_test[df_test["time_since_order"] > end_time]
    test_sets[f"{j} months"] = df_test

test_sets

{'0 months':                                order_id                customer_unique_id  \
 59023  98e54035aaf4e7a4b0464f9e6066febf  12869eae016893325456717ed2b57427   
 43779  70aefd12bdd5391f02011072c16e6801  297b0df27a85a8c6c3ba6600a4142d52   
 85022  daaf62a15bd3c9f093e0e1f58413c8c2  972d898cec7230c1d435b308078620b6   
 68183  b001a9f70c34cbf4eca4e48875de711c  771c1a711afde25be37e7a59f24e5919   
 53584  8a7096900fa8a063219b9edd1a9e5431  3e4ef963d1c04ababbd4cf03c4d034a6   
 ...                                 ...                               ...   
 94889  f44117f1303c4e6573d463bdedff85de  e07bc32087f23e0d27ff1cfb337943d8   
 8504   15f29653eecf60489d5e04389f448526  d69251081d9edc5fbdbe3fb96df96105   
 21906  38878bafc676263af50211706ab39e7c  ab93b5411d105681f97717263e92d917   
 95108  f4db4affcb95da2a38c0713f619c8313  dc9d7738c8da323923a5205895d5b398   
 41046  69b5f9421e21822b212fa48bf807b811  88894f059c62f3b3030445e7a73323c0   
 
        total_payments  mean_review_score  time_si

In [97]:
n_months = [2, 3, 6]
bin_edges = dict()

for n in n_months:
    n_periods = (train_data["time_since_order"].max() - pd.Timedelta(60)).days // (30 * n)
    indices = []

    for i in range(1, n_periods):
        last_order = train_data["time_since_order"][
            train_data["time_since_order"] <= pd.Timedelta(n * (i + 1) * 30, unit="D")
        ].iloc[-1]
        indices.append(last_order)
    bin_edges[f"{n} months"] = indices

print(bin_edges)



{'2 months': [Timedelta('119 days 23:58:04'), Timedelta('179 days 23:54:07'), Timedelta('239 days 23:59:34'), Timedelta('299 days 23:47:25'), Timedelta('359 days 23:57:20'), Timedelta('419 days 23:56:27'), Timedelta('479 days 23:42:17'), Timedelta('539 days 23:46:44'), Timedelta('599 days 23:58:16'), Timedelta('650 days 05:34:12'), Timedelta('662 days 18:13:31')], '3 months': [Timedelta('179 days 23:54:07'), Timedelta('269 days 23:56:07'), Timedelta('359 days 23:57:20'), Timedelta('449 days 23:59:16'), Timedelta('539 days 23:46:44'), Timedelta('629 days 23:54:54'), Timedelta('662 days 18:13:31')], '6 months': [Timedelta('359 days 23:57:20'), Timedelta('539 days 23:46:44'), Timedelta('662 days 18:13:31')], '12 months': [Timedelta('662 days 18:13:31')]}


In [265]:
simulation_data = dict()

for n in n_months:



for n in n_months:
    print(n_months, "months: ", end="")


    df_with_periods = prdz.fit_transform(df)

    df_agg_period = df_with_periods.groupby("period_id")
    print(df_with_periods)
    periods = dict()
    csum = CustomerSummary()
    cstd = CustomerStandardizer()

    for pid in range(1, df_with_periods["period_id"].max() + 1):
        periods[pid] = (
            cstd.fit_transform(
                csum.fit_transform(
                    df_agg_period.get_group(pid).drop("period_id", axis=1))))

    simulation_data[str(n_months) + " months"] = periods

    print("--------------------------")

2 months: 12 bins created.
                               order_id                customer_unique_id  \
6511   10a045cdf6a5650c21e9cfeb60384c16  87ab9fec999db8bd5774917de3cdf01c   
68317  b059ee4de278302d550a3035c4cdb740  262e1f1e26e92e86375f86840b4ffd63   
62892  a2ac6dad85cf8af5b0afb510a240fe8c  af5454198a97379394cacf676e1e96cb   
37735  616fa7d4871b87832197b2a137a115d2  634420a0ea42302205032ed44ac7fccc   
22190  392ed9afd714e3c74767d0c4d3e3f477  9bb92bebd4cb7511e1a02d5e50bc4655   
...                                 ...                               ...   
43966  71303d7e93b399f5bcd537d124c0bcfa  0eb1ee9dba87f5b36b4613a65074337c   
74448  bfbd0f9bdef84302105ad712db648a6c  830d5b7aaa3b6f1e9ad63703bec97d23   
49869  809a282bbd5dbcabb6f2f724fca862ec  009b0127b727ab0ba422f6d9604487c7   
89489  e5fa5a7210941f7d56d0208e4e071d35  4854e9b3feff728c13ee5fc7d1547e92   
17931  2e7a8482f6fb09756ca50c10d7bfc047  b7d76e111c89f7ebf14761390f0f7d17   

       total_payments  mean_review_score        

In [263]:
print(simulation_data["2 months"][3])

                                  n_orders  total_payments  mean_review_score  \
customer_unique_id                                                              
0000f46a3911fa3c0805444483337064 -0.124636       -0.299959          -0.779368   
0005e1862207bf6ccc02e4228effd9a0 -0.124636        0.350708          -0.035377   
0011805441c0d1b68b48002f1d005526 -0.124636        1.155089          -0.779368   
00191a9719ef48ebb5860b130347bf33 -0.124636       -0.745572          -0.779368   
001f3c4211216384d5fe59b041ce1461 -0.124636       -1.320219          -0.779368   
...                                    ...             ...                ...   
ffba9f9dff87b05e310ecc46c8591044 -0.124636        3.164525           0.708613   
ffe254cc039740e17dd15a5305035928  7.400730       -0.385789          -0.779368   
ffe2dd1f3b0cbf0b5f35e818ec03c49b -0.124636       -0.007106          -0.035377   
ffe3e199b9d0b7fb7d2d29a5b9498447 -0.124636        0.441019          -2.267349   
ffedff0547d809c90c05c2691c51

In [260]:
kmeans_model = KMeans(n_clusters=5, random_state=42)

for key in list(simulation_data.keys()):
    scores = []
    for i in range(1, len(simulation_data[key]) + 1):
        data = simulation_data[key][i]
        print(data)
        clusters = kmeans_model.fit_predict(data)
        silhouette_avg = silhouette_score(data, clusters)
        scores.append(silhouette_avg)
    print(key, ":", scores)

                                  n_orders  total_payments  mean_review_score  \
customer_unique_id                                                              
281096eb031de8c318bba51915fdd16a       0.0             0.0                0.0   

                                  time_since_order  mean_delay  
customer_unique_id                                              
281096eb031de8c318bba51915fdd16a               0.0         0.0  


ValueError: n_samples=1 should be >= n_clusters=5.